# What is Chatbot?
* A chatbot is an intelligent piece of software that is capable of communicating and performing actions similar to a human.
* Chatbots are used a lot in customer interaction, marketing on social network sites and instantly messaging the client
* Ther are two basic types of chatbot models based on how they are built; Retrieval based and Generative based models.

# 1. Retrieval based Chatbots
* A retrieval-based chatbot uses predefined input patterns and responses.
* It then uses some type of heuristic approach to select the appropriate responses.
* It is widely used in the industry to make goal-oriented chatbots where we can customize the tone and flow ot the chatbot to drive our customers with the best experience.

# 2. Generative based Chatbots
* Generative models are not based on some predefined responses.
* They are based on seq 2 seq neural networks. 
* It is the same idea as machine translation
* In machine translation, we translate the source code from one language to another language 
* But, here we are going to transform input into an output. It needs a large amount of data and it is based on Deep neural networks

# Project Detail
* We are going to build chatbot using deep learning techniques
* The chatbot will be trained on the dataset which contains categories(intents), pattern and responses
* We use a special recurrent neural networl (LSTM) to classify which category the user's message belongs to and then we will give a random response from the list of responses.

# The Dataset
The dataset we will be using is 'intents.json'. This is a JSON file that contains the patterns we need to find and the responses we want to return to the user

# Perequisites
* Keras and Natural Language Processing (NLTK)

# Steps
1. Import and load the data file
2. Preprocess data
3. Create training and testing data
4. Build the model
5. Predict the response

# 1. Import and load the data file
* Import necessary packages and initialize the variables
* Use the json package to parse the JSON file into puthon

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

Using TensorFlow backend.


In [8]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('dhusi.json').read()
intents = json.loads(data_file)

In [9]:
#nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ace\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Preprocess data
* When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model.
* Tokenizing is the most basic and first thing you can do on text data.
* Tokenizing is the process of breaking the whole text into small parts like words.
* Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function
* And append each word in the words list
* We also create a list of classes for our tags.

In [10]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


* Now we will lemmatize each word and remove duplicate words from the list
* Lemmatizing is the process of converting a word into its lemma from and then creating a pickle file to store the python objects which we will use while predicting

In [11]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

39 documents
9 classes ['address', 'age', 'food', 'goodbye', 'greeting', 'hobbies', 'name', 'study', 'thanks']
59 unique lemmatized words ["'s", ',', 'a', 'address', 'age', 'anyone', 'are', 'awesome', 'bye', 'call', 'chatting', 'day', 'do', 'favourite', 'food', 'for', 'free', 'good', 'goodbye', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hobby', 'hola', 'home', 'how', 'i', 'in', 'is', 'later', 'level', 'like', 'live', 'me', 'name', 'next', 'nice', 'normally', 'old', 'pooja', 'see', 'should', 'student', 'study', 'studying', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'whats', 'where', 'you', 'your']


# 3. Create training and testing data

* Now, we will create the training data in which we will provide the input and the output
* Our input will be the pattern and output will be the class our input pattern belongs to
* But the computer doesn't understand text so we will convert text into numbers


In [12]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


# 4. Build the model

* we have our training data ready
* Now we will build a deep neural network that has 3 layers
* We use the Keras sequential API for this
* After training the model for 200 epochs
* We achieved 100% accurcay on our model
* Let us save the model as "chatbot_model.h5'

In [13]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('dhusi_bot.h5', hist)

print("model created")

Epoch 1/200
39/39 [==============================] - 0s 2ms/step - loss: 2.2071 - accuracy: 0.0769
Epoch 2/200
39/39 [==============================] - 0s 308us/step - loss: 2.2212 - accuracy: 0.1538
Epoch 3/200
39/39 [==============================] - 0s 282us/step - loss: 2.0491 - accuracy: 0.2564
Epoch 4/200
39/39 [==============================] - 0s 282us/step - loss: 2.1454 - accuracy: 0.1282
Epoch 5/200
39/39 [==============================] - 0s 332us/step - loss: 2.0675 - accuracy: 0.3846
Epoch 6/200
39/39 [==============================] - 0s 333us/step - loss: 1.9271 - accuracy: 0.3333
Epoch 7/200
39/39 [==============================] - 0s 334us/step - loss: 1.8692 - accuracy: 0.3590
Epoch 8/200
39/39 [==============================] - 0s 308us/step - loss: 1.9154 - accuracy: 0.4359
Epoch 9/200
39/39 [==============================] - 0s 257us/step - loss: 1.7870 - accuracy: 0.4872
Epoch 10/200
39/39 [==============================] - 0s 281us/step - loss: 1.8051 - accuracy

39/39 [==============================] - 0s 308us/step - loss: 0.0837 - accuracy: 0.9744
Epoch 82/200
39/39 [==============================] - 0s 282us/step - loss: 0.0620 - accuracy: 1.0000
Epoch 83/200
39/39 [==============================] - 0s 256us/step - loss: 0.0328 - accuracy: 1.0000
Epoch 84/200
39/39 [==============================] - 0s 256us/step - loss: 0.0809 - accuracy: 0.9744
Epoch 85/200
39/39 [==============================] - 0s 256us/step - loss: 0.0519 - accuracy: 1.0000
Epoch 86/200
39/39 [==============================] - 0s 231us/step - loss: 0.0889 - accuracy: 1.0000
Epoch 87/200
39/39 [==============================] - 0s 282us/step - loss: 0.1091 - accuracy: 0.9744
Epoch 88/200
39/39 [==============================] - 0s 256us/step - loss: 0.0583 - accuracy: 1.0000
Epoch 89/200
39/39 [==============================] - 0s 205us/step - loss: 0.0488 - accuracy: 1.0000
Epoch 90/200
39/39 [==============================] - 0s 256us/step - loss: 0.0472 - accuracy: 

39/39 [==============================] - 0s 282us/step - loss: 0.0298 - accuracy: 1.0000
Epoch 161/200
39/39 [==============================] - 0s 307us/step - loss: 0.0503 - accuracy: 1.0000
Epoch 162/200
39/39 [==============================] - 0s 333us/step - loss: 0.0234 - accuracy: 1.0000
Epoch 163/200
39/39 [==============================] - 0s 257us/step - loss: 0.0891 - accuracy: 0.9487
Epoch 164/200
39/39 [==============================] - 0s 282us/step - loss: 0.0534 - accuracy: 0.9744
Epoch 165/200
39/39 [==============================] - 0s 333us/step - loss: 0.0146 - accuracy: 1.0000
Epoch 166/200
39/39 [==============================] - 0s 308us/step - loss: 0.0753 - accuracy: 0.9744
Epoch 167/200
39/39 [==============================] - 0s 231us/step - loss: 0.0549 - accuracy: 1.0000
Epoch 168/200
39/39 [==============================] - 0s 256us/step - loss: 0.0396 - accuracy: 0.9744
Epoch 169/200
39/39 [==============================] - 0s 333us/step - loss: 0.0612 - a